In [1]:
from data_prep import one_hot_decode, get_bars_dataset
import numpy as np
from predictions import define_models,predict_sequence
from music21 import converter, instrument, note, chord, stream, midi, environment
%matplotlib inline

from midi_handling import noteArrayToStream, switch_range
from music21 import *
environment.set("musescoreDirectPNGPath", "/usr/bin/musescore")

Using TensorFlow backend.


In [2]:
# Only for GPU use:
#os.environ["CUDA_VISIBLE_DEVICES"]="1"


import keras.backend as K
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

In [3]:
# configure problem
# configure problem
weights_path = "weights/512_64/weights-improvement-2455-1.00.hdf5"

MELODY_NOTE_OFF = 128 # (stop playing all previous notes)
MELODY_NO_EVENT = 129 # (no change from previous event)

# configure problem
n_features = 131
cardinality = 131
timesteps = 16

learning_rate = 0.0001

n_encoder_units = 512
n_decoder_units = n_encoder_units
latent_dim = 64

dropout=0.3

beta=1
epochs = 20000
batch_size = 64
num_training_samples = 10000
num_validation_samples = 1000
steps_per_epoch = int(num_training_samples / batch_size)
validation_steps = int(num_validation_samples / batch_size)

start_of_sequence = np.zeros(cardinality)
start_of_sequence[cardinality-1] = 1

In [4]:
# define model
train, infenc, infdec = define_models(n_encoder_units=n_encoder_units,
                                        n_decoder_units=n_decoder_units,
                                        latent_dim=latent_dim, 
                                      timesteps=timesteps,
                                      n_features=n_features,
                                      learning_rate=learning_rate,
                                        dropout=dropout,
                                      beta=beta,
                                        epsilon_std=1.)

## Set weights

In [5]:
train.load_weights(weights_path)
print("Loaded weights from disk")

all_weights = train.get_weights()
decoder_position = len(infenc.get_weights())
decoder_weights = all_weights[decoder_position::]
encoder_weights = all_weights[:decoder_position]

infenc.set_weights(encoder_weights)
infdec.set_weights(decoder_weights)

Loaded weights from disk


### Predict a sequence

In [6]:
# Datasets
np.random.seed(10)
test_ID_list_dict = []
ID_list = np.load("ID_lists/test_ID_list.npy")

test_list = ID_list.item().get("test_labels")

input_songs = []
output_songs = []

ID = test_list[500]
encoder_input_before = np.load('encoder_inputs/' + ID + '.npy')
encoder_input = encoder_input_before.reshape(1, timesteps, cardinality)    
encoder_output = predict_sequence(infenc=infenc,
                                  infdec=infdec,
                                  source=encoder_input,
                                  seq_len=160,
                                  timesteps=timesteps,
                                  n_decoder_units=n_decoder_units,
                                  latent_dim=latent_dim,
                                  temperature=1.5)

input_songs.append(one_hot_decode(encoder_input_before))
output_songs.append(one_hot_decode(encoder_output))

In [6]:
output_songs = np.array(output_songs).flatten()

melody_stream = noteArrayToStream(switch_range(list(np.ravel(output_songs))))

#melody_stream.show()

In [7]:
switch_range(np.ravel(output_songs))

array([129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
       129,  30, 129, 129, 129,  40, 129, 128, 129, 129, 129, 129, 129,
       129, 129, 128, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
       129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
       129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
       129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
       129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
       129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
       129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
       129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
       129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
       129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,
       129, 129, 129, 129])

### Predict a song

In [ ]:
#np.random.seed(10)

song_from_file = np.load("songs_encoder_inputs/id-2000.npy")
song_len = song_from_file.shape[0]

# reshaping to work as input to lstm
song = np.array(song_from_file).reshape(song_len, timesteps, cardinality)

orig_song = []
pred_song = []
    
for bar in song:
    encoder_input = bar.reshape(1, timesteps, cardinality)
    encoder_output = predict_sequence(infenc=infenc,
                                  infdec=infdec,
                                  source=encoder_input,
                                  seq_len=16,
                                  timesteps=16,
                                  latent_dim=latent_dim,
                                  temperature=1.2)

    orig_song.append(one_hot_decode(encoder_input.reshape(timesteps, cardinality)))
    pred_song.append(one_hot_decode(encoder_output.reshape(timesteps, cardinality)))

In [ ]:
len(orig_song), len(pred_song)

In [ ]:
np.ravel(switch_range(pred_song))

In [ ]:
#np.isclose(orig_song, pred_song, rtol=0.1)

melody_stream = noteArrayToStream(switch_range(list(np.ravel(pred_song))))

melody_stream.show()